In [1]:
!python -m torch.utils.collect_env

PyTorch version: 2.1.1
Is debug build: False
CUDA used to build PyTorch: 12.1
ROCM used to build PyTorch: N/A

OS: Microsoft Windows 11 Pro
GCC version: Could not collect
Clang version: Could not collect
CMake version: Could not collect
Libc version: N/A

Python version: 3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)] (64-bit runtime)
Python platform: Windows-10-10.0.22631-SP0
Is CUDA available: True
CUDA runtime version: Could not collect
CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: GPU 0: NVIDIA GeForce RTX 4090
Nvidia driver version: 546.29
cuDNN version: Could not collect
HIP runtime version: N/A
MIOpen runtime version: N/A
Is XNNPACK available: True

CPU:
Architecture=9


CurrentClockSpeed=4201


DeviceID=CPU0


Family=107


L2CacheSize=16384


L2CacheSpeed=


Manufacturer=AuthenticAMD


MaxClockSpeed=4201


Name=AMD Ryzen 9 7950X3D 16-Core Processor          


ProcessorType=3


Revision=24834

Versions of relevant libraries:
[pip3] numpy==1.26.

In [28]:
import pandas as pd

train_data = pd.read_csv('seed.csv')
test_data = pd.read_csv('test.csv')

# Check the first few rows of the data
print(train_data.head())
print(test_data.head())

                                             example        label
0  Seller: Hello Buyer: Wow love the Couch. SO lo...    furniture
1  Seller: I am selling this for $28500. Buyer: I...          car
2  Buyer: I'd like to negotiate a lower price for...      housing
3  Seller: Hi!  Are you interested in my headphon...  electronics
4  Seller: Hi. Were you interested in the mirror?...    furniture
                                             example      label
0  Seller: Hi are you interested in buying my Pin...  furniture
1  Buyer: Hello I am interested in your property ...    housing
2  Buyer: Hello . How long have you owned the dre...  furniture
3  Buyer: I am very interested place you have for...    housing
4  Buyer: Hey, nice car you have here, how long h...        car


In [29]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data):
    return tokenizer(data['example'].tolist(), padding=True, truncation=True, return_tensors='pt')

train_inputs = tokenize_data(train_data)
test_inputs = tokenize_data(test_data)

In [30]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train_labels = label_encoder.fit_transform(train_data['label'])
test_labels = label_encoder.transform(test_data['label'])

In [31]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Move the model to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [33]:
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

train_labels = torch.tensor(train_labels, dtype=torch.long).to(device)

train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()

for epoch in range(3):  # You may need to adjust the number of epochs
    model.train()
    for batch in train_loader:
        inputs, attention_mask, labels = batch
        inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


In [34]:
model.eval()

with torch.no_grad():
    test_inputs = {key: val.to(device) for key, val in test_inputs.items()}
    outputs = model(**test_inputs)
    predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Evaluate accuracy or other metrics
accuracy = (predictions == test_labels).mean()
print(f"Accuracy: {accuracy}")


Accuracy: 0.835
